In [66]:
import model
import torch
import dataset
import model
import math
import numpy as np
import utility
import os


In [67]:
def test_reg_model(model, test_dataset, batch_size=50, criterion=None, num_rects=4, screen_w=640, screen_h=480):
    model.eval()
    
    if criterion is None:
        criterion = torch.nn.MSELoss().cuda()

    print("CURRENT MODEL seq_len: {}".format(test_dataset.seq_len))
    print("CURRENT MODEL: {}".format(model.__class__.__name__))
    print("CURRENT DATASET SIZE: {}".format(test_dataset.__len__()))
    print("CURRENT NUM RECTS: {}".format(num_rects))
    
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                               batch_size=batch_size,
                                               shuffle=True)

    min_error = 999999
    max_error = 0
    sum_error = 0
    count_error = 0
    errors = np.zeros(test_dataset.__len__())

    correct = 0
    total = 0
    
    min_loss = 999999
    max_loss = 0
    sum_loss = 0
    count_loss = 0

    with torch.no_grad():
        for i, (eye_left, eye_right, face, pos) in enumerate(test_loader):
            eye_left = eye_left.to(device)
            eye_right = eye_right.to(device)
            face = face.to(device)
            pos = pos.to(device)

            # Forward pass
            out = model(eye_left, eye_right, face)
            pos = pos[:, -1, :]
            loss = criterion(out, pos)

            out = out.cpu().detach().numpy()
            pos = pos.cpu().numpy()

            for b in range(pos.shape[0]):
                o = out[b]
                p = pos[b]

                error = math.sqrt((o[0] - p[0])**2 + (o[1] - p[1])**2)

                if error < min_error:
                    min_error = error
                if error > max_error:
                    max_error = error
                sum_error += error

                errors[count_error] = error
                
                o_x = math.ceil((o[0]) / (screen_w / num_rects))
                o_y = math.ceil((o[1]) / (screen_h / num_rects))

                p_x = math.ceil((p[0]) / (screen_w / num_rects))
                p_y = math.ceil((p[1]) / (screen_h / num_rects))

                
                if ((p_y - 1) * num_rects + (p_x - 1)) == ((o_y - 1) * num_rects + (o_x - 1)):
                    correct += 1
                    
                total += 1
                
                count_error += 1

            if loss.item() < min_loss:
                min_loss = loss.item()
            if loss.item() > max_loss:
                max_loss = loss.item()
            sum_loss += loss.item()
            count_loss += 1

        print('MIN MSELoss: {} '.format(min_loss))
        print('MAX MSELoss: {} '.format(max_loss))
        print('AVG MSELoss: {}'.format(sum_loss / count_loss))

        print('MIN error: {} '.format(min_error))
        print('MAX error: {} '.format(max_error))
        print('AVG error: {}'.format(sum_error / count_error))
        print('MEAD error: {}'.format(np.median(errors)))

    print('Accuracy : %d %%' % (100 * correct / total))
    return test_dataset.__len__(), min_error, max_error, (sum_error / count_error), (np.median(errors)), errors


In [75]:
def test_classifire_model(model, test_dataset, batch_size=50):
    model.eval()  
    
    criterion = torch.nn.BCELoss().cuda()

    print("CURRENT MODEL seq_len: {}".format(test_dataset.seq_len))
    print("CURRENT MODEL: {}".format(model.__class__.__name__))
    print("CURRENT DATASET SIZE: {}".format(test_dataset.__len__()))
    print("CURRENT NUM RECTS: {}".format(num_rects))

    test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                               batch_size=batch_size,
                                               shuffle=True)

    correct = 0
    total = 0

    min_loss = 999999
    max_loss = 0
    sum_loss = 0
    count_loss = 0

    with torch.no_grad():
        for i, (eye_left, eye_right, face, pos) in enumerate(test_loader):
            eye_left = eye_left.to(device)
            eye_right = eye_right.to(device)
            face = face.to(device)
            pos = pos.to(device)

            # Forward pass
            out = model(eye_left, eye_right, face)
            loss = criterion(out, pos)

            _, predicted = torch.max(out.data, 1)
            _, pos = torch.max(pos.data, 1)
            total += pos.size(0)
            
            correct += (predicted == pos).sum().item()
            if loss.item() < min_loss:
                min_loss = loss.item()
            if loss.item() > max_loss:
                max_loss = loss.item()
            sum_loss += loss.item()
            count_loss += 1

        
        print('MIN BCELoss: {} '.format(min_loss))
        print('MAX BCELoss: {} '.format(max_loss))
        print('AVG BCELoss: {}'.format(sum_loss / count_loss))
        
    print('Accuracy : %d %%' % (100 * correct / total))

In [91]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

dataset_seq_len = 60

batch_size = 10

num_rects = 8
seq_len = 8

max_samples = 2500 


In [92]:
import model
import torch
import dataset
import model
import math
import numpy as np
import utility
import os

model = utility.load_model('./models/EyeClassifierLSTM/model_{}_{}_{}.pth'.format(
    max_samples, num_rects * num_rects, seq_len), 
                           device,
                           model.EyeClassifierLSTM(num_rects * num_rects, seq_len))

dataset = dataset.CDataset(num_rects=num_rects, seq_len=seq_len, dirname='./test')

test_classifire_model(model, dataset, batch_size)

FOUND MODEL


./test/60/
1178
LOADING DATA...


0
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


END LOAD DATA


1000
[28. 18. 19. 27. 17. 17. 16. 18. 16. 27.  9. 15. 18. 20. 46. 20. 15. 20.
  5.  7.  7.  7. 13. 16. 12. 15.  5.  4. 16.  5. 23. 14. 23. 15.  8.  4.
  8. 11. 19. 14. 15. 18. 10.  5. 16. 21. 22. 15. 14. 18. 15. 11.  9. 16.
 13.  8. 20. 19. 15. 17. 18. 16. 12. 41.]
[36. 23. 21. 28. 27. 20. 18. 21. 19. 28.  9. 17. 20. 23. 54. 28. 17. 21.
  8. 14.  7. 10. 15. 19. 13. 17.  7.  6. 23.  7. 28. 19. 30. 15. 10.  5.
 12. 14. 20. 14. 15. 20. 12.  5. 18. 21. 24. 17. 15. 19. 19. 11. 11. 17.
 15. 11. 29. 20. 17. 18. 21. 17. 15. 48.]
CURRENT MODEL seq_len: 8
CURRENT MODEL: EyeClassifierLSTM
CURRENT DATASET SIZE: 1178
CURRENT NUM RECTS: 8
MIN BCELoss: 0.034604866057634354 
MAX BCELoss: 0.10874444246292114 
AVG BCELoss: 0.06646370322648752
Accuracy : 23 %


In [93]:
import model
import torch
import dataset
import model
import math
import numpy as np
import utility
import os

model = utility.load_model('./models/TwoEyesLSTM/model_{}_{}.pth'.format(
    max_samples, seq_len), 
            device,
            model.TwoEyesLSTM(seq_len=seq_len))

dataset = dataset.Dataset(seq_len=seq_len, dirname='./test')

test_reg_model(model, dataset, batch_size=batch_size, num_rects=num_rects)

FOUND MODEL


./test/60/
1178
LOADING DATA...


0


END LOAD DATA


1000
CURRENT MODEL seq_len: 8
CURRENT MODEL: TwoEyesLSTM
CURRENT DATASET SIZE: 1178
CURRENT NUM RECTS: 8
MIN MSELoss: 1200.9395751953125 
MAX MSELoss: 32570.88671875 
AVG MSELoss: 6028.128250380694
MIN error: 1.8478840698735317 
MAX error: 765.4087950207258 
AVG error: 88.51248956705753
MEAD error: 74.16208318192423
Accuracy : 18 %


(1178,
 1.8478840698735317,
 765.4087950207258,
 88.51248956705753,
 74.16208318192423,
 array([142.94261932,  78.53524034,  60.13605643, ...,  25.5049737 ,
         73.2134301 ,  48.0855381 ]))